#### 해결해야할 문제1) 0시 이후 시간을 -1하다보니 -1된 상품이 해당 일의 가장 먼저 방송된 것으로 처리됨.
#### 예) 9월 15일 01:20에 방송된 쥐포는 9월 14일의 마지막 방송으로 처리되어야 하지만 9월 14일의 첫 방송으로 처리됨.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from datetime import datetime
# pd.set_option('display.max_rows', 100)
%matplotlib inline
plt.style.use('ggplot') # 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.rcParams['font.family'] = 'Malgun Gothic'

In [77]:
# 나눔고딕코드 폰트 설치 (필요한사람만 돌리세요)
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# plt.rc('font', family='NanumBarunGothic')

In [95]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    from google.colab import drive
    drive.mount('/content/drive')
    sum_df = pd.read_csv('/content/drive/My Drive/빅데이터_콘테스트/sum_df.csv')
else:
    print('Not running on CoLab')
    sum_df = pd.read_csv('c:/bc/sum_df.csv')

Not running on CoLab


In [96]:
sum_df[sum_df['HOUR'] == 0].HOUR = 24
sum_df[sum_df['HOUR'] == 1].HOUR = 25
sum_df[sum_df['HOUR'] == 2].HOUR = 26

C:\Users\user\anaconda3\envs\mulcam\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
sum_df['quarter'] = pd.to_datetime(sum_df.broadcast).dt.quarter

In [98]:
sum_df['timeslot'] = pd.cut(sum_df.HOUR, bins = [0,8,11,14,17,20,23,26], \
                             labels = [1,2,3,4,5,6,7])

In [99]:
seasons = [4, 4, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4]
month_to_season = dict(zip(range(1,13), seasons))

In [101]:
sum_df['season'] = sum_df.MONTH.map(month_to_season)

In [104]:
sum_df

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,...,DAY_NAME,DAY_NUM,p_group_code,new_code,no,sales_level,grade,quarter,timeslot,season
0,2019-01-01 06:00:00,60.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,32696000.0,820.0,2019,...,Tuesday,2,0,13.0,0,b,4,1,1,4
1,2019-01-01 07:00:00,60.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,47878000.0,812.0,2019,...,Tuesday,2,1,7.0,6,b,4,1,1,4
2,2019-01-01 08:00:00,60.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,99736000.0,1665.0,2019,...,Tuesday,2,0,16.0,9,d,3,1,1,4
3,2019-01-01 09:00:00,60.0,100816,202404,보코 리버시블 무스탕,의류,79000,90973000.0,1151.0,2019,...,Tuesday,2,0,18.0,12,c,3,1,2,4
4,2019-01-01 10:00:00,60.0,100809,202395,CERINI by PAT 남성 풀패키지 기모니트 3종,의류,79900,259678000.0,3250.0,2019,...,Tuesday,2,0,17.0,15,e,2,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,2019-12-31 18:20:00,60.0,100356,201134,마르엘라로사티 에코무스탕1종,의류,79000,201322000.0,2548.0,2019,...,Tuesday,2,0,8745.0,38259,e,2,4,5,4
6964,2019-12-31 19:20:00,60.0,100594,201819,레스토닉 블루투스 LED 서랍 침대 슈퍼싱글,가구,599000,102360000.0,155.0,2019,...,Tuesday,2,10,8723.0,38262,a,3,4,5,4
6965,2019-12-31 20:20:00,60.0,100501,201516,무이자 국내제조 노비타 뉴스파 비데 무료설치,생활용품,219000,191710000.0,762.0,2019,...,Tuesday,2,6,8741.0,38271,b,2,4,5,4
6966,2019-12-31 21:20:00,60.0,100155,200556,일시불 LG전자 매직스페이스 냉장고,가전,1359000,222022000.0,157.0,2019,...,Tuesday,2,5,8725.0,38283,a,2,4,6,4


### 공휴일 정보 가져오기(펜딩)

In [ ]:
# import requests

# def get_request_query(url, operation, params, serviceKey):
#     import urllib.parse as urlparse
#     params = urlparse.urlencode(params)
#     request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
#     return request_query

# # 요청 URL과 오퍼레이션
# URL = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
# OPERATION = 'getHoliDeInfo' # 국경일 + 공휴일 정보 조회 오퍼레이션

# # 파라미터
# SERVICEKEY = 'ufpQyXdzW5U3fKtDjkxrQU%2B072wtNzOh3olKrC4j8HwY3N93fKkf2CaStzl2siNVZsfzdODW7GWHR8Cyj1UgwQ%3D%3D'
# solYear  = '2020'  # 연도
# solMonth = '09'   # 월
# PARAMS = {'solYear':solYear, 'solMonth':solMonth}

# request_query = get_request_query(URL, OPERATION, PARAMS, SERVICEKEY)
# print('request_query:', request_query)
# response = requests.get(url=request_query)
# print('status_code:' + str(response.status_code))

# if True == response.ok:
#     print(response.text)